In [1]:
from typing import List

**22. Generate Parentheses**


In [4]:
class Solution:  # 83% time, 64% memory
    def generateParenthesis(self, n: int) -> List[str]:
        # 2 options at most: add open or add close
        res = []
        stack = []

        def dfs(open: int, close: int):
            # open is remaining number of open available
            # same for close
            if close == 0:  # base case
                res.append("".join(stack))
                return

            if open > 0:
                stack.append("(")
                dfs(open - 1, close)
                stack.pop()
            if close > 0 and close > open:
                # there must always be more open brackets placed
                stack.append(")")
                dfs(open, close - 1)
                stack.pop()

        dfs(n, n)
        return res


generateParenthesis = Solution()
generateParenthesis.generateParenthesis(3)

['((()))', '(()())', '(())()', '()(())', '()()()']

**739. Daily Temperatures**


In [9]:
class Solution:  # 68% time, 96% memory
    def dailyTemperatures(self, temperatures: List[int]) -> List[int]:
        # will traverse backwards because I only need to consider the sorted max values in front of a given index
        res = [0 for _ in range(len(temperatures))]
        stack = [(temperatures[-1], len(temperatures) - 1)]  # store value and position
        for i in range(len(temperatures) - 2, -1, -1):
            while stack and temperatures[i] >= stack[-1][0]:
                stack.pop()
            if stack:
                res[i] = stack[-1][1] - i

            stack.append((temperatures[i], i))
        return res

    def dailyTemperaturesForward(self, temperatures: list[int]) -> list[int]:
        res = [0 for _ in range(len(temperatures))]
        stack = []
        for i in range(len(temperatures)):
            while (
                stack and temperatures[i] > stack[-1][0]
            ):  # stack is in descending order in terms of temperatures
                _, old_i = stack.pop()
                res[old_i] = i - old_i
            stack.append((temperatures[i], i))
        return res


dailyTemperatures = Solution()
dailyTemperatures.dailyTemperatures([73, 74, 75, 71, 69, 72, 76, 73])
dailyTemperatures.dailyTemperaturesForward([73, 74, 75, 71, 69, 72, 76, 73])

[1, 1, 4, 2, 1, 1, 0, 0]

**853. Car Fleet**


In [16]:
class Solution:
    def carFleet(self, target: int, position: List[int], speed: List[int]) -> int:
        # a stack is a fleet, when fleet is determined clear stack and increment `res`
        # starting from leftmost car since its the most limiting factor
        # make an array of completion times for each car: time 't' when start position + speed * 't' = target
        cars = sorted(zip(position, speed))
        stack = []
        for i in range(len(cars) - 1, -1, -1):
            # use stack and delete cars that collide into ones in front of them
            stack.append((target - cars[i][0]) / cars[i][1])
            if len(stack) > 1 and stack[-1] <= stack[-2]:
                stack.pop()
        return len(stack)

    def carFleetAlso(self, target: int, position: List[int], speed: List[int]) -> int:
        cars = list(zip(position, speed))
        cars.sort()  # traverse backwards (start with car furthest ahead)
        finish = [(target - p) / s for p, s in cars]
        # if car reaches target faster or equal than car in front,
        # do not add it to stack because it forms a fleet
        # otherwise add to stack because it is a new fleet
        stack = [finish[-1]]  # only finish times will be kept
        for i in range(len(finish) - 2, -1, -1):
            if finish[i] > stack[-1]:
                stack.append(finish[i])
        return len(stack)


target = 10
position = [0, 4, 2]
speed = [2, 1, 3]

carFleet = Solution()
carFleet.carFleet(target, position, speed)

1

**2969. Minimum String Length After Removing Substrings**


In [2]:
class Solution:  # 73% time, 82% memory
    def minLength(self, s: str) -> int:
        stack = []
        for c in s:
            if stack and stack[-1] + c in ["AB", "CD"]:
                stack.pop()
            else:
                stack.append(c)
        return len(stack)


S = Solution()
S.minLength("ABFCACDB")

2

**84. Largest Rectangle in Histogram**


In [41]:
class Solution:  # 21% time, 92% memory
    def largestRectangleArea(self, heights: List[int]) -> int:
        """
        Just watch NeetCode's video. I am unable to rephrase it into a docstring
        """
        res = 0

        # will always be monotonic increasing, heights[j] > heights[i] for j < i will be popped from the stack
        # pair: (heights[i], j) where j is the leftmost index with a value >= to heights[i]
        # j is stored that way so that you can compute the largest surface area extending leftwards and rightwards
        stack = []
        for i in range(len(heights)):
            j = i
            res = max(res, heights[i])
            # print(stack)
            while stack and heights[i] <= stack[-1][0]:
                height_j, j = stack.pop()
                # now that you have finally found a height of smaller height than `height_j`
                #   (means you cannot keep extending it rightwards)
                # you can calculate the surface of a rectangle starting at `j` and ending at `i-1`
                # area_j = height_j * ((i-1) - j + 1)
                #   `+ 1` since a single data point is treated as width of 1
                res = max(res, height_j * (i - j))
            # append the in
            stack.append((heights[i], j))

        # compute the area of the heights remaining in the stack which can be extended to the right entirely
        # note that the stack is monotonic increasing which is why this works:
        # all heights remaining are allowed to extend all the way to the end of the array
        #   thanks to the fact that there is a height at the end taller than them.
        for height, i in stack:
            res = max(res, height * (len(heights) - i))

        return res


largestRectangleArea = Solution()
for lst in [
    [2, 1, 5, 6, 2, 3],
    [2, 4],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8, 9, 7, 1],
    [1, 1],
    [2, 3],
]:
    print(largestRectangleArea.largestRectangleArea(lst))

10
4
21
2
4


**2116. Check if a Parentheses String Can Be Valid**


In [8]:
class Solution:  # 98% time, 9% memory
    def canBeValid(self, s: str, locked: str) -> bool:
        """
        Keep two stacks
        - One for locked "(" characters
        - One for wildcards

        At the end you need to go through the stack of locked "(" and ensure that there is a wildcard to the right for each of them.
        """
        if len(s) % 2:
            # need an even length
            return False

        # will store the indices to make that second iteration easy
        unlocked = []
        l_locked = []
        for i in range(len(s)):
            # Can restructure these if statements to remove one layer of nesting
            # if locked[i] == "0": unlocked.append(i), and the use elif statement
            if locked[i] == "1":
                if s[i] == ")":
                    if l_locked:
                        l_locked.pop()
                    elif unlocked:
                        unlocked.pop()
                    else:
                        return False
                else:
                    l_locked.append(i)
            else:
                unlocked.append(i)

        for i in reversed(l_locked):
            if unlocked and unlocked[-1] > i:
                unlocked.pop()
            else:
                return False

        return True


canBeValid = Solution()
print(canBeValid.canBeValid("))()))", "010100"))
print(canBeValid.canBeValid("()", "11"))

[] [4, 5]
True
[] []
True
